## Import the Necessary Libraries

In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim         
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2024-03-18 15:58:55.422777: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 15:58:55.441854: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 15:58:55.441874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 15:58:55.442343: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 15:58:55.445819: I tensorflow/core/platform/cpu_feature_guar

cpu


## Loading the Preprocessed Test Dataset

In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

# Read data from CSV files
notes_df = pd.read_csv('/home/admin1/Desktop/NEWLSTM/csv_dataset/notes.csv')
test_df = pd.read_csv('/home/admin1/Desktop/NEWLSTM/csv_dataset/testset.csv')

# Extract data and labels from the test set
data_test = test_df[['x_test', 'future']].to_numpy()
x_test_string = data_test[:, 0]
y_test_string = data_test[:, 1]

# Convert string representations to lists
x_test = []
y_test = []

for i in x_test_string:
    # Remove square brackets and newline characters
    i = i.strip('[]\n')
    # Split the string and convert elements to integers
    input_x_test = [int(j) for j in i.split()]
    x_test.append(input_x_test)

for i in y_test_string:
    # Remove square brackets and newline characters
    i = i.strip('[]\n')
    # Split the string and convert elements to integers
    input_y_test = [int(j) for j in i.split()]
    y_test.append(input_y_test)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Create a dictionary of unique notes
notes_ = notes_df.iloc[:, 0].to_numpy()
unique_notes = dict(enumerate(notes_.flatten(), 0))

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
class MusicDataset(Dataset):
    def __init__(self, data , data_labels):
        self.data = data
        self.data_labels = data_labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx] , self.data_labels[idx]


test_set = MusicDataset(x_test,y_test)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1,
                                         shuffle=False, num_workers=2)

## Load the Trained Model

In [4]:
Net = torch.load('/home/admin1/Desktop/NEWLSTM/state_dictionary/lstmmodel_basic.pth')
print(Net)

OrderedDict([('embedding.weight', tensor([[-0.9660, -0.1753, -0.9998,  ..., -2.4478,  0.7413,  0.1131],
        [ 1.1218, -0.3555, -1.0031,  ...,  0.2686,  0.4533, -1.3828],
        [ 0.6009,  0.6317,  1.8816,  ..., -0.2737, -0.3908,  1.7856],
        ...,
        [ 0.0391,  2.5164,  2.5064,  ...,  0.6519, -0.4782, -0.4989],
        [-0.2807,  0.6004,  1.6323,  ...,  0.9294,  0.1259,  0.5602],
        [-0.0171, -0.7937,  0.0736,  ..., -1.7346, -1.5824, -2.2183]])), ('lstm.weight_ih_l0', tensor([[ 0.2736, -0.5099,  0.2236,  ..., -0.1463,  1.0522, -0.2698],
        [ 0.0973,  0.1648,  0.4945,  ..., -0.2400,  0.9486, -0.4667],
        [ 0.3904,  0.2224, -0.2705,  ...,  0.0825,  0.2308, -0.8683],
        ...,
        [-0.0575, -0.3865, -0.0631,  ..., -0.0593,  0.0908, -0.1832],
        [ 0.3537, -0.2953, -0.4213,  ..., -0.6575,  0.0952,  0.4562],
        [-0.1336,  0.3779, -0.3101,  ...,  1.0342,  0.2886,  0.1842]])), ('lstm.weight_hh_l0', tensor([[ 0.2977,  0.0255, -0.7435,  ..., -0.1524,

## Initialize the same model as that in Training

In [5]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, num_classes):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        
        self.embedding = nn.Embedding(num_classes, 100)
        self.lstm = nn.LSTM(input_size=100, hidden_size=256, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Take the last timestep's output
        x = self.fc1(lstm_out)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

## Evaluate the model based on Test Data

In [8]:
import torch
import numpy as np
from torch.utils.data import DataLoader

num_classes = 301
seq_length = 100

model = LSTM(num_classes)  # Instantiate your model
model.load_state_dict(torch.load('/home/admin1/Desktop/NEWLSTM/state_dictionary/lstmmodel_basic.pth'))  # Load the state dictionary

# Define testloader
testloader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)

total_preds = 0
correct_preds = 0
future_preds = 8

for i, data in enumerate(testloader, 0):
    input, label = data
    cumm_output = torch.zeros(0, len(unique_notes)).to(device)
    cumm_label = np.array([], dtype=int)
    
    for k in range(future_preds):
        # Perform inference using the model
        output = model(input.to(device))
        cumm_output = torch.cat((cumm_output, output))
        cumm_label = np.concatenate((cumm_label, label[:, k]))
        next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
        total_preds += input.shape[0]
        correct_preds += torch.sum(torch.argmax(output, 1) == label[:, k].to(device))
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
                                           for ind, j in enumerate(input)])[:, 1:])

test_acc = float(correct_preds) / float(total_preds) * 100
testreport = "Testing Accuracy : \n correct predictions  : {} \n total predictions : {} \n Testing Accuracy : {} \n ------------------------\n".format(correct_preds, total_preds, test_acc)
print(testreport)

Testing Accuracy : 
 correct predictions  : 6823 
 total predictions : 108568 
 Testing Accuracy : 6.284540564438877 
 ------------------------



## Saving the generated output in the form of MIDI

In [9]:
def note_to_int(note):
    # Dictionary mapping note names to MIDI numbers
    note_dict = {
        'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3, 'E': 4, 'F': 5,
        'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8, 'Ab': 8, 'A': 9, 'A#': 10, 'Bb': 10, 'B': 11
    }
    if isinstance(note, str):  # if note is a string128N
        if note.isdigit():  # if note is a number string
            return int(note)
        else:  # if note is a note string
            # Separate the pitch and the octave (e.g. 'E5' -> ('E', '5'))
            pitch, octave = note[:-1], note[-1]
            # Replace '-' with 'b' in the pitch
            pitch = pitch.replace('-', 'b')
            # Calculate the MIDI number
            midi_num = note_dict[pitch] + (int(octave) + 1) * 12
            return midi_num
    elif isinstance(note, int):  # if note is an integer
        return note
    else:
        raise ValueError(f"Invalid note: {note}")

In [10]:
import mido

def convert_to_midi(prediction_output, path):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            for current_note in notes_in_chord:
                cn = note_to_int(current_note)
                # Create note_on and note_off events
                track.append(mido.Message('note_on', note=cn, velocity=64, time=0))
                track.append(mido.Message('note_off', note=cn, velocity=64, time=480))  # Assuming note length of 480 ticks
        # pattern is a note
        else:
            cn = note_to_int(pattern)
            # Create note_on and note_off events
            track.append(mido.Message('note_on', note=cn, velocity=64, time=0))
            track.append(mido.Message('note_off', note=cn, velocity=64, time=480))  # Assuming note length of 480 ticks

    mid.save(path)
    print(f"MIDI file saved at: {path}")

In [12]:
import os
import random  # Import the random module

for j in range(10):
    index = random.randint(0,len(x_test))
    print("Taking the seed tune as follows:")
    print(x_test[index])
    tune = x_test[index]
    input = np.empty((1,32),dtype=int)
    input[0] = tune
    input = torch.from_numpy(input)
    next_preds = 64
    for i in range(next_preds):
        #output = Net(input.to(device),input.shape[0])
        output = model(input.to(device))
        next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
                                                   for ind,j in enumerate(input)])[:,1:]) 

        tune = np.insert(tune,-1,next_preds[0])
    tune = [unique_notes[i] for i in tune]
    path = '/home/admin1/Desktop/NEWLSTM/Output/music'+str(j)+'.midi'
    
    # Check if the directory exists, if not, create it
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Try to write the MIDI file
    try:
        convert_to_midi(tune, path)
        print(f"MIDI file saved at: {path}")
    except Exception as e:
        print(f"Failed to write MIDI file: {e}")

Taking the seed tune as follows:
[156 233 175   6 142 230  65 230 156 191 191  39  67 215 255  29  57  91
 230 211 215  44  39 155 131 263   8 211 211 156  67 143]
MIDI file saved at: /home/admin1/Desktop/NEWLSTM/Output/music0.midi
MIDI file saved at: /home/admin1/Desktop/NEWLSTM/Output/music0.midi
Taking the seed tune as follows:
[126 281  31 236 281 164  31 280   2  90  16 174  90  31 281   2 286  16
  36 280 137 259  31 286  55  90  31 185 281 166  90  55]
MIDI file saved at: /home/admin1/Desktop/NEWLSTM/Output/music1.midi
MIDI file saved at: /home/admin1/Desktop/NEWLSTM/Output/music1.midi
Taking the seed tune as follows:
[219 287  34  84 186  84   1 192 155 171 155  41 218  66 235 235 125  91
 269  82 122  82 122 211  46  46  57  13 122 142 290 142]
MIDI file saved at: /home/admin1/Desktop/NEWLSTM/Output/music2.midi
MIDI file saved at: /home/admin1/Desktop/NEWLSTM/Output/music2.midi
Taking the seed tune as follows:
[125 164 231  49  49 100 231 242 100 231 100 293 293 220 279  92 10